In [7]:
import math
import re
from datetime import datetime

In [23]:
#define a Class that stores the overall state when being changed by each operation
#in here define all gate operations on qubit state
class State():
    v = math.sqrt(2) / 2
    #initialize state with n, this can be easily changed in the state
    def __init__(self, n):
        #create a list on which qubits tensor will act upon
        self.states = []
        #create a list and append each state to the list, here all states are initiliazed to zero
        qubits = []
        #append qubit states in tensor list
        for i in range(n):
            qubits.append(Qubit(0))
        self.states.append(QubitTensor(qubits))
        
    #define X gate for masterstate class, tidx gives index where X gate applied on
    def X(self, tidx):
        self.states[tidx].X()
        
    #Define H gate 
    def H(self, tidx, qidx):
        qubitTensor = self.states[tidx]
        qubit = qubitTensor.qubits[qidx]
        wasOne = qubit.bit == 1
        qubitTensor.qubits[qidx].bit = 0 
        
    #each hadamard splits array into two subarrays
    #we create a new qubitTensor list that saves the new subarrays
    #we add the new amplitudes
        qt = QubitTensor([])
        for i, qubit in enumerate(qubitTensor.qubits):
            if i != qidx:
                qt.append(Qubit(qubit.bit))
            else:
                qt.append(Qubit(1))

        qubitTensor.coeff *= (1 / math.sqrt(2))
        qt.coeff = qubitTensor.coeff * (-1 if wasOne else 1)

        self.states.insert(tidx + 1, qt)

    #define CNOT gate
    def CX(self, tidx, control, target):
        qubitTensor = self.states[tidx]
        qubit = qubitTensor.qubits[control]
        #if the qubit is 1, apply CNOT only, and apply X on target index
        if qubit.bit == 1:
            qubitTensor.X(target)

    #define general T gate
    def T(self, tidx, qidx):
        qubitTensor = self.states[tidx]
        qubit = qubitTensor.qubits[qidx]
        # applies only when qubit is one
        if qubit.bit == 1:
            #only sign change for amplitude
            qubitTensor.coeff *= (self.v + self.v * 1j)

    #define T dagger, same as T       
    def D(self, tidx, qidx):
        qubitTensor = self.states[tidx]
        qubit = qubitTensor.qubits[qidx]
        if qubit.bit == 1:
            qubitTensor.coeff *= (self.v - self.v * 1j)
            
   #define representation helper function, that outputs final state
    def __repr__(self):
        s = ''
        for t in self.states:
            state = ''
            for q in t.qubits:
                state += str(q.bit)
            state = f"{t.coeff}|{state}>"
        
            s += str(state) + '\n'
        return s

In [24]:
#define qubit operations as qubit tensor, define state, operations and coefficient
class QubitTensor():
    def __init__(self, qubits=[], op=None, coeff=1):
        self.qubits = qubits
        self.op = op
        self.coeff = coeff

    # define x gate, idx defines which qubit is acted upon
    def X(self, idx):
        self.qubits[idx].bit = int(not self.qubits[idx].bit)

    # define helper function append to append new qubit states
    def append(self, qubit):
        self.qubits.append(qubit)

    # define helper function equation for simplification of state
    def __eq__(self, other):
        for i in range(len(self.qubits)):
            if self.qubits[i].bit != other.qubits[i].bit:
                return False
        return True

#define qubit state as 0 or 1
class Qubit():
    def __init__(self,bit):
        self.bit = bit

In [25]:
#define simplify function to reduce state after Hadamard has been executed
#namely sort according to bit pattern and combine states with same pattern
def simplify(State):
    count = 0
    while count < len(State.states)*2:
        #remove numbers where imaginary coefficient is zero and where real part is very close to zero
        for i, qubitTensor in enumerate(State.states):
            imag = qubitTensor.coeff.imag
            real = qubitTensor.coeff.real
            if (imag == 0) and (0.0000000001 >= real >= -0.0000000001):
                trash = i
                State.states = State.states[:trash] + State.states[trash + 1:]
                break
            #compare states and combine amplitudes if same state
            #all other states then remove
            #once all states found break
            found = False
            for j, qubitTensor2 in enumerate(State.states):
                if i == j:
                    continue

                if qubitTensor == qubitTensor2:
                    qubitTensor.coeff += qubitTensor2.coeff

                    trash = j
                    State.states = State.states[:trash] + State.states[trash + 1:]
                    found = True
                    break
            if found:
                break
        count += 1

In [26]:
def simulate(qubitCount):
    #open selected file
    f = open('rd84_142.qasm')
    #initialize the state with 16 qubits calling class MasterState

    finalState = State(qubitCount)
    #read through each file from line 4 onwards
    for i, line in enumerate(f.readlines()[4:]):

        #find all target and control values
        idx = [int(x) for x in re.findall(r'\d+', line)]

        instr = line[:line.find(' ')]
        #look for each gate and apply from class Masterstate
        if instr == 'x':
            for tensor in finalState.states:
                tensor.X(idx[0])
        elif instr == 'cx':
            for i, tensor in enumerate(finalState.states):
                finalState.CX(i, idx[0], idx[1])
        elif instr == 'h':
            k = len(finalState.states)*2
            for i in range(0, k, 2):
                finalState.H(i, idx[0])
            #to reduce runtime we use simplify function after each Hadamard expression if possible
            simplify(finalState)
        elif instr == 't':
            for i, tensor in enumerate(finalState.states):
                finalState.T(i, idx[0])
        elif instr == 'tdg':
            for i, tensor in enumerate(finalState.states):
                finalState.D(i, idx[0])

    #simplify state again at the end to get one final state
    simplify(finalState)

    print('FINAL RESULT')
    print(finalState)
    
    
    
def main():
    f = open('rd84_142.qasm')
    k = f.readlines()[3]
    n = [int(x) for x in re.findall(r'\d+', k)][0]
    simulate(n)
  

In [27]:
%%time
main()

FINAL RESULT
(1+0j)|0000000000000000>

CPU times: user 7.34 ms, sys: 6.42 ms, total: 13.8 ms
Wall time: 39.7 ms
